In [1]:
from utils import get_embedding, LlamaEmbeddingFunction
import os

In [2]:
chapter_files = sorted(os.listdir('./data/harry_potter/chapters'))
chapter_ids = list(map(lambda x: x.split('.')[0].replace(" ", "_"), chapter_files[:17]))

In [ ]:
# load each chapter and get the embedding
embeddings = []
for chapter_file in chapter_files[:17]:
    chapter = open(f'data/harry_potter/chapters/{chapter_file}', 'r').read()
    embedding = get_embedding(chapter)
    embeddings.append(embedding)

In [3]:
#load all chapters into memory 
chapters = []
for chapter_file in chapter_files[:17]:
    chapter = open(f'data/harry_potter/chapters/{chapter_file}', 'r').read()
    chapters.append(chapter)

In [4]:

# and get the embedding

embedding_f = LlamaEmbeddingFunction()


In [5]:

embeddings = embedding_f(chapters)

--------

ChromaDB

In [7]:
import chromadb

In [8]:
client = chromadb.PersistentClient(path="chroma/harry_potter_chapters")

In [9]:
collection = client.create_collection(name="harry_potter_chapters", embedding_function=embedding_f, metadata={"hnsw:space": "cosine"})
collection = client.get_collection(name="harry_potter_chapters", embedding_function=embedding_f)


In [10]:
metadatas = []
for id in chapter_ids:
    c_number = id.split("_")[0]
    c_name = " ".join(id.split("_")[1:])
    metadatas.append({"chapter_number": c_number, "chapter_name": c_name})

In [ ]:

collection.add(embeddings=embeddings, metadatas=metadatas, ids=chapter_ids)

In [11]:
collection.add(
    documents=chapters,
    metadatas=metadatas,
    ids=chapter_ids,
)

-------

Querying

In [12]:
question = "Who imagines socks when looking at the mirror of erised ?"
q_embedding = get_embedding(question)
collection.query(
    query_embeddings=q_embedding,
    n_results=5,
    include=["documents", "distances"],
    #where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"sock"}
)

{'ids': [['002_the_vanashig_glass',
   '003_the_letters_from_no_one_the_escape_of_the_brazilian_boa_constrictor_earned_harry_his_longest_ever_punishment',
   '012_the_mirror_of_erised',
   '010_halloween']],
 'distances': [[0.918466861363467,
   0.9589774892521882,
   0.9782705682512174,
   1.030559930988758]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['Chapter : THE VANASHIG GLASS .\n Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step , but Privet Drive had hardly changed at all .\n The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys ’ front door it crept into their living room , which was almost exactly the same as it had been on the night when Mr Dursley had seen that fateful news report about the owls .\n Only the photographs on the mantelpiece really showed how much time had passed .\n Ten years ago , there had been lots of pictures of what looked like a large pink beach ball we

In [ ]:
collection.